In [1]:
import pandas as pd
import geopandas as gp
import shutil
import os
import numpy as np
from math import sqrt

#Set options for pandas displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## VEST Documentation

Election results from Alaska Division of Elections (http://www.elections.alaska.gov/results/18GENR/index.php)
Precinct shapefile from Alaska Division of Elections (http://www.elections.alaska.gov/Core/districtmaps.php)

Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD. Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.

G18GOVRDUN - Mike Dunleavy (Republican Party)
G18GOVDBEG - Mark Begich (Democratic Party)
G18GOVLTOI - William Toien (Libertarian Party)
G18GOVOWRI - Write-in Votes

G18HALRYOU - Don Young (Republican Party)
G18HALDGAL - Alyse S. Galvin (Democratic Party)
G18HALOWRI - Write-in Votes

Not mentioned in VEST documentation but *is* included in their file

***G18GOVIWAL*** - Bill Walker (Independent)

## Import all necessary files

In [2]:
vest = gp.read_file('./raw-sources/ak_2018/ak_2018.shp')

elec = pd.read_csv('./raw-sources/resultsbyprecinct.txt',header=None)

p = gp.read_file('./raw-sources/2013-SW-Precinct-Proc-Plan/SW Proc Shape Files/2013-SW-Proc-Shape-files.shp')

In [3]:
display(vest.head())
display(elec.head())
display(p.head())

,DISTRICT,NAME,G18GOVRDUN,G18GOVDBEG,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALRYOU,G18HALDGAL,G18HALOWRI,geometry
0,11-075,11-075 Palmer City No. 2,655,352,28,14,3,632,407,5,"POLYGON ((-149.13365 61.61026, -149.13361 61.6..."
1,12-220,12-220 Butte,1442,528,37,30,1,1426,588,13,"POLYGON ((-148.52103 61.62682, -148.52020 61.6..."
2,10-055,10-055 Lakes No. 2,453,155,19,6,3,450,178,1,"POLYGON ((-149.33012 61.61391, -149.33229 61.6..."
3,15-300,15-300 JBER No. 2,201,112,8,12,1,170,161,2,"POLYGON ((-149.93126 61.25090, -149.92104 61.2..."
4,28-110,28-110 Rabbit Creek No. 2,1016,792,14,17,0,1030,812,4,"POLYGON ((-149.83082 61.07593, -149.83055 61.0..."


,0,1,2,3,4,5,6
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,Total,1,NaN
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,Total,1,NaN
2,01-446 Aurora,US REPRESENTATIVE,Number of Precincts for Race,NP,Total,1,NaN
3,01-446 Aurora,US REPRESENTATIVE,Number of Precincts Reporting,NP,Total,1,NaN
4,01-446 Aurora,US REPRESENTATIVE,Registered Voters,NP,Total,2727,NaN


,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry
0,173,1.565101,11-075,1.0,None,11-075 Palmer City No. 2,2460,1622.0,838.0,0.516646,"POLYGON ((-149.13365 61.61026, -149.13361 61.6..."
1,181,286.270233,12-220,1.0,None,12-220 Butte,3990,1622.0,2368.0,1.459926,"POLYGON ((-148.52103 61.62682, -148.52020 61.6..."
2,182,3.675227,10-055,1.0,None,10-055 Lakes No. 2,1325,1622.0,-297.0,-0.183107,"POLYGON ((-149.33012 61.61391, -149.33229 61.6..."
3,32,29.413515,15-300,1.0,None,15-300 JBER No. 2,5937,1622.0,4315.0,2.660296,"POLYGON ((-149.93126 61.25090, -149.92104 61.2..."
4,144,6.259140,28-110,1.0,None,28-110 Rabbit Creek No. 2,2785,1622.0,1163.0,0.717016,"POLYGON ((-149.83082 61.07593, -149.83055 61.0..."


## Process the election data

In [4]:
# Look at the data

elec.head()

,0,1,2,3,4,5,6
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,Total,1,NaN
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,Total,1,NaN
2,01-446 Aurora,US REPRESENTATIVE,Number of Precincts for Race,NP,Total,1,NaN
3,01-446 Aurora,US REPRESENTATIVE,Number of Precincts Reporting,NP,Total,1,NaN
4,01-446 Aurora,US REPRESENTATIVE,Registered Voters,NP,Total,2727,NaN


In [5]:
# Create a combined race/candidate field to pivot on

elec['race_cand'] = elec.apply(lambda x: '_'.join([x[1],x[2]]),axis=1)
elec.head()

,0,1,2,3,4,5,6,race_cand
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,Total,1,NaN,Race Statistics _Number of Precincts
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,Total,1,NaN,Race Statistics _Number of Precincts Reporting
2,01-446 Aurora,US REPRESENTATIVE,Number of Precincts for Race,NP,Total,1,NaN,US REPRESENTATIVE _Number of Precincts for Race
3,01-446 Aurora,US REPRESENTATIVE,Number of Precincts Reporting,NP,Total,1,NaN,US REPRESENTATIVE _Number of Precincts Reporting
4,01-446 Aurora,US REPRESENTATIVE,Registered Voters,NP,Total,2727,NaN,US REPRESENTATIVE _Registered Voters


In [6]:
# Pivot the election data 

piv = elec.pivot_table(index = [0], columns = ['race_cand'])
piv = piv.fillna(0)

piv.head()

5  \
race_cand               Ballot Measure No. 1 - 17FSH2 _NO    
0                                                            
01-446 Aurora                                        570.0   
01-455 Fairbanks No. 1                                99.0   
01-465 Fairbanks No. 2                               167.0   
01-470 Fairbanks No. 3                               325.0   
01-475 Fairbanks No. 4                               145.0   

                                                                                       \
race_cand               Ballot Measure No. 1 - 17FSH2 _Number of Precincts Reporting    
0                                                                                       
01-446 Aurora                                                          1.0              
01-455 Fairbanks No. 1                                                 1.0              
01-465 Fairbanks No. 2                                                 1.0              
01-470 Fairbanks No. 3                                                 1.0              
01-475 Fairbanks No. 4                                                 1.0              

                                                                                      \
race_cand               Ballot Measure No. 1 - 17FSH2 _Number of Precincts for Race    
0                                                                                      
01-446 Aurora                                                          1.0             
01-455 Fairbanks No. 1                                                 1.0             
01-465 Fairbanks No. 2                                                 1.0             
01-470 Fairbanks No. 3                                                 1.0             
01-475 Fairbanks No. 4                                                 1.0             

                                                                           \
race_cand               Ballot Measure No. 1 - 17FSH2 _Registered Voters    
0                                                                           
01-446 Aurora                                                      2727.0   
01-455 Fairbanks No. 1                                              502.0   
01-465 Fairbanks No. 2                                             1246.0   
01-470 Fairbanks No. 3                                             1567.0   
01-475 Fairbanks No. 4                                              782.0   

                                                                       \
race_cand               Ballot Measure No. 1 - 17FSH2 _Times Counted    
0                                                                       
01-446 Aurora                                                   868.0   
01-455 Fairbanks No. 1                                          155.0   
01-465 Fairbanks No. 2                                          284.0   
01-470 Fairbanks No. 3                                          493.0   
01-475 Fairbanks No. 4                                          246.0   

                                                             \
race_cand               Ballot Measure No. 1 - 17FSH2 _YES    
0                                                             
01-446 Aurora                                         274.0   
01-455 Fairbanks No. 1                                 53.0   
01-465 Fairbanks No. 2                                100.0   
01-470 Fairbanks No. 3                                152.0   
01-475 Fairbanks No. 4                                 92.0   

                                                   \
race_cand               Dis Court JD1 Miller _NO    
0                                                   
01-446 Aurora                                 0.0   
01-455 Fairbanks No. 1                        0.0   
01-465 Fairbanks No. 2                        0.0   
01-470 Fairbanks No. 3                        0.0   
01-475 Fairbanks No. 4                        0.0   

          

In [7]:
# Create a list of candidate last names and potential races

cand_names = ['Dunleavy','Begich','Toien','Write-in','Young','Galvin','Walker/Mallott']
opts = ['GOVERNOR','REPRESENTATIVE']

In [8]:
# Match candidate names to actual names as recorded in the election results

cols_keep = []
for i in piv.columns:
    for j in cand_names:
        if str(j) in str(i):
            if str(j)=='Write-in':
                for k in opts:
                    if str(k) in str(i):
                        cols_keep.append(i)
                    else:
                        continue
            else:
                cols_keep.append(i)
print(cols_keep)

[(5, 'GOVERNOR/LT. GOVERNOR _Begich/Call '), (5, 'GOVERNOR/LT. GOVERNOR _Dunleavy/Meyer '), (5, 'GOVERNOR/LT. GOVERNOR _Toien/Clift '), (5, 'GOVERNOR/LT. GOVERNOR _Walker/Mallott '), (5, 'GOVERNOR/LT. GOVERNOR _Write-in '), (5, 'US REPRESENTATIVE _Galvin, Alyse S. '), (5, 'US REPRESENTATIVE _Write-in '), (5, 'US REPRESENTATIVE _Young, Don ')]


In [9]:
# Subset the pivo to the columns desired.

piv = piv[cols_keep]
piv.head()

5  \
race_cand               GOVERNOR/LT. GOVERNOR _Begich/Call    
0                                                             
01-446 Aurora                                         399.0   
01-455 Fairbanks No. 1                                 65.0   
01-465 Fairbanks No. 2                                124.0   
01-470 Fairbanks No. 3                                253.0   
01-475 Fairbanks No. 4                                121.0   

                                                                \
race_cand               GOVERNOR/LT. GOVERNOR _Dunleavy/Meyer    
0                                                                
01-446 Aurora                                            427.0   
01-455 Fairbanks No. 1                                    84.0   
01-465 Fairbanks No. 2                                   139.0   
01-470 Fairbanks No. 3                                   203.0   
01-475 Fairbanks No. 4                                   110.0   

                                                             \
race_cand               GOVERNOR/LT. GOVERNOR _Toien/Clift    
0                                                             
01-446 Aurora                                          29.0   
01-455 Fairbanks No. 1                                  3.0   
01-465 Fairbanks No. 2                                 13.0   
01-470 Fairbanks No. 3                                 16.0   
01-475 Fairbanks No. 4                                  9.0   

                                                                \
race_cand               GOVERNOR/LT. GOVERNOR _Walker/Mallott    
0                                                                
01-446 Aurora                                             11.0   
01-455 Fairbanks No. 1                                     3.0   
01-465 Fairbanks No. 2                                     4.0   
01-470 Fairbanks No. 3                                     9.0   
01-475 Fairbanks No. 4                                     5.0   

                                                          \
race_cand               GOVERNOR/LT. GOVERNOR _Write-in    
0                                                          
01-446 Aurora                                        0.0   
01-455 Fairbanks No. 1                               0.0   
01-465 Fairbanks No. 2                               1.0   
01-470 Fairbanks No. 3                               3.0   
01-475 Fairbanks No. 4                               1.0   

                                                              \
race_cand               US REPRESENTATIVE _Galvin, Alyse S.    
0                                                              
01-446 Aurora                                          429.0   
01-455 Fairbanks No. 1                                  66.0   
01-465 Fairbanks No. 2                                 142.0   
01-470 Fairbanks No. 3                                 265.0   
01-475 Fairbanks No. 4                                 136.0   

                                                      \
race_cand               US REPRESENTATIVE _Write-in    
0                                                      
01-446 Aurora                                    9.0   
01-455 Fairbanks No. 1                           0.0   
01-465 Fairbanks No. 2                           1.0   
01-470 Fairbanks No. 3                           1.0   
01-475 Fairbanks No. 4                           2.0   

                                                        
race_cand               US REPRESENTATIVE _Young, Don   
0                                                       
01-446 Aurora                                    419.0  
01-455 Fairbanks No. 1                            88.0  
01-465 Fairbanks No. 2                           136.0  
01-470 Fairbanks No. 3                           223.0  
01-475 Fairbanks No. 4                           104.0

In [10]:
# Reset the index and add as a column.

piv = piv.reset_index()

piv.head()

0                                   5  \
race_cand                          GOVERNOR/LT. GOVERNOR _Begich/Call    
0                   01-446 Aurora                                399.0   
1          01-455 Fairbanks No. 1                                 65.0   
2          01-465 Fairbanks No. 2                                124.0   
3          01-470 Fairbanks No. 3                                253.0   
4          01-475 Fairbanks No. 4                                121.0   

                                                  \
race_cand GOVERNOR/LT. GOVERNOR _Dunleavy/Meyer    
0                                          427.0   
1                                           84.0   
2                                          139.0   
3                                          203.0   
4                                          110.0   

                                               \
race_cand GOVERNOR/LT. GOVERNOR _Toien/Clift    
0                                        29.0   
1                                         3.0   
2                                        13.0   
3                                        16.0   
4                                         9.0   

                                                  \
race_cand GOVERNOR/LT. GOVERNOR _Walker/Mallott    
0                                           11.0   
1                                            3.0   
2                                            4.0   
3                                            9.0   
4                                            5.0   

                                            \
race_cand GOVERNOR/LT. GOVERNOR _Write-in    
0                                      0.0   
1                                      0.0   
2                                      1.0   
3                                      3.0   
4                                      1.0   

                                                                             \
race_cand US REPRESENTATIVE _Galvin, Alyse S.  US REPRESENTATIVE _Write-in    
0                                        429.0                          9.0   
1                                         66.0                          0.0   
2                                        142.0                          1.0   
3                                        265.0                          1.0   
4                                        136.0                          2.0   

                                          
race_cand US REPRESENTATIVE _Young, Don   
0                                  419.0  
1                                   88.0  
2                                  136.0  
3                                  223.0  
4                                  104.0

In [11]:
# Add the precinct column name as a new column

loc = 0
for i in piv.columns:
    if loc == 0:
        piv[('','NAME')] = piv[i]
        loc = 1

piv.head()

0                                   5  \
race_cand                          GOVERNOR/LT. GOVERNOR _Begich/Call    
0                   01-446 Aurora                                399.0   
1          01-455 Fairbanks No. 1                                 65.0   
2          01-465 Fairbanks No. 2                                124.0   
3          01-470 Fairbanks No. 3                                253.0   
4          01-475 Fairbanks No. 4                                121.0   

                                                  \
race_cand GOVERNOR/LT. GOVERNOR _Dunleavy/Meyer    
0                                          427.0   
1                                           84.0   
2                                          139.0   
3                                          203.0   
4                                          110.0   

                                               \
race_cand GOVERNOR/LT. GOVERNOR _Toien/Clift    
0                                        29.0   
1                                         3.0   
2                                        13.0   
3                                        16.0   
4                                         9.0   

                                                  \
race_cand GOVERNOR/LT. GOVERNOR _Walker/Mallott    
0                                           11.0   
1                                            3.0   
2                                            4.0   
3                                            9.0   
4                                            5.0   

                                            \
race_cand GOVERNOR/LT. GOVERNOR _Write-in    
0                                      0.0   
1                                      0.0   
2                                      1.0   
3                                      3.0   
4                                      1.0   

                                                                             \
race_cand US REPRESENTATIVE _Galvin, Alyse S.  US REPRESENTATIVE _Write-in    
0                                        429.0                          9.0   
1                                         66.0                          0.0   
2                                        142.0                          1.0   
3                                        265.0                          1.0   
4                                        136.0                          2.0   

                                                                   
race_cand US REPRESENTATIVE _Young, Don                      NAME  
0                                  419.0           01-446 Aurora   
1                                   88.0  01-455 Fairbanks No. 1   
2                                  136.0  01-465 Fairbanks No. 2   
3                                  223.0  01-470 Fairbanks No. 3   
4                                  104.0  01-475 Fairbanks No. 4

In [12]:
# Drop the first level of the tuple in column names

piv.columns = piv.columns.droplevel(0)

piv.head()

race_cand,,GOVERNOR/LT. GOVERNOR _Begich/Call,GOVERNOR/LT. GOVERNOR _Dunleavy/Meyer,GOVERNOR/LT. GOVERNOR _Toien/Clift,GOVERNOR/LT. GOVERNOR _Walker/Mallott,GOVERNOR/LT. GOVERNOR _Write-in,"US REPRESENTATIVE _Galvin, Alyse S.",US REPRESENTATIVE _Write-in,"US REPRESENTATIVE _Young, Don",NAME
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4


In [13]:
# Rename the columns according to VEST structure

pc = list(piv.columns)

rename_dict = {pc[1]:'G18GOVDBEG',
              pc[2]:'G18GOVRDUN',
              pc[3]:'G18GOVLTOI',
              pc[4]:'G18GOVIWAL',
              pc[5]:'G18GOVOWRI',
              pc[6]:'G18HALDGAL',
              pc[7]:'G18HALOWRI',
              pc[8]:'G18HALRYOU'}
piv.rename(columns=rename_dict,inplace=True)

piv.head()

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4


In [14]:
# Look at data in full

display(piv)

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4
5,01-480 Fairbanks No. 5,321.0,336.0,14.0,7.0,0.0,312.0,2.0,361.0,01-480 Fairbanks No. 5
6,01-485 Fairbanks No. 6,177.0,173.0,9.0,4.0,0.0,197.0,2.0,166.0,01-485 Fairbanks No. 6
7,01-490 Fairbanks No. 7,207.0,190.0,19.0,10.0,2.0,220.0,1.0,205.0,01-490 Fairbanks No. 7
8,01-495 Fairbanks No. 10,84.0,107.0,3.0,3.0,0.0,86.0,2.0,108.0,01-495 Fairbanks No. 10
9,02-345 Badger No. 2,249.0,679.0,25.0,16.0,0.0,286.0,7.0,671.0,02-345 Badger No. 2


In [15]:
# Rename the one statewide precinct to fit the structure of others

piv['NAME'] = piv['NAME'].apply(lambda x: x.replace('HD99 Fed Overseas Absentee','HD99-Absentee'))

In [16]:
# Create a 'DISTRICT' field that takes the numeric description of the precinct

piv['DISTRICT'] = piv['NAME'].apply(lambda x:'-'.join([ x.split(' ')[1], x.split('-')[1]]) if x.startswith('District') else x.split(' ')[0])

piv.head()

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME,DISTRICT
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora,01-446
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1,01-455
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2,01-465
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3,01-470
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4,01-475


In [17]:
# Create a field that associates the precinct with the Alaska House District

piv['HD'] = piv['DISTRICT'].apply(lambda x: x.split('-')[0])
piv['HD'] = piv['HD'].apply(lambda x: x.zfill(2))

piv.head()

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME,DISTRICT,HD
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora,01-446,01
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1,01-455,01
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2,01-465,01
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3,01-470,01
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4,01-475,01


In [18]:
# Create a textual description field 

piv['DESCRIPTION'] = piv['NAME'].apply(lambda x: (x.split('-')[1]).strip())

piv.head()

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME,DISTRICT,HD,DESCRIPTION
0,01-446 Aurora,399.0,427.0,29.0,11.0,0.0,429.0,9.0,419.0,01-446 Aurora,01-446,01,446 Aurora
1,01-455 Fairbanks No. 1,65.0,84.0,3.0,3.0,0.0,66.0,0.0,88.0,01-455 Fairbanks No. 1,01-455,01,455 Fairbanks No. 1
2,01-465 Fairbanks No. 2,124.0,139.0,13.0,4.0,1.0,142.0,1.0,136.0,01-465 Fairbanks No. 2,01-465,01,465 Fairbanks No. 2
3,01-470 Fairbanks No. 3,253.0,203.0,16.0,9.0,3.0,265.0,1.0,223.0,01-470 Fairbanks No. 3,01-470,01,470 Fairbanks No. 3
4,01-475 Fairbanks No. 4,121.0,110.0,9.0,5.0,1.0,136.0,2.0,104.0,01-475 Fairbanks No. 4,01-475,01,475 Fairbanks No. 4


In [19]:
# Subset the statewide precinct to be its own dataframe

overseas = piv[piv['HD']=='HD99'].copy()

display(overseas)

race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME,DISTRICT,HD,DESCRIPTION
561,HD99 Fed Overseas Absentee,0.0,0.0,0.0,0.0,0.0,55.5,0.0,3.5,HD99-Absentee,HD99-Absentee,HD99,Absentee


In [20]:
# Remove the overseases/statewide precinct from the pivoted election data

piv = piv[~piv['HD'].isin(['HD99'])].copy()

In [21]:
# Determine if the fields are unique and the difference between the number of precincts in the datasets.

not_matching = set(set(list(p['DISTRICT'])) ^ set(list(piv['DISTRICT'])))
dset_diff = abs(len(p) - len(piv))
print('Number of mismatching DISTRICTs: ', len(not_matching))
print('Length of election dataset: ', len(piv))
print('Number of unique IDs in the election dataset: ', len(list(piv['DISTRICT'].unique())))

match = str(len(list(piv['DISTRICT'].unique()))==len(piv))
print('Is the ID unique in the election dataset? (T/F) ', match)
print('Length of precinct dataset: ', len(p))
print('Number of unique IDs in the precinct shapefile: ', len(list(p['DISTRICT'].unique())))
match = str(len(list(p['DISTRICT'].unique()))==len(p))
print('Is the ID unique in the election dataset? (T/F) ', match)
print('Length of VEST dataset: ', len(vest))
print('Number of unique IDs in the VEST dataset: ', len(list(vest['DISTRICT'].unique())))
match = str(len(list(vest['DISTRICT'].unique()))==len(vest))
print('Is the ID unique in the election dataset? (T/F) ', match)
print('Difference in length of precinct shapefile and election data: ', dset_diff)
match = str(dset_diff==len(not_matching))
print('Is the difference between the two datasets the same as missing values? (T/F)', match)
print(not_matching)

Number of mismatching DISTRICTs:  120
Length of election dataset:  561
Number of unique IDs in the election dataset:  561
Is the ID unique in the election dataset? (T/F)  True
Length of precinct dataset:  441
Number of unique IDs in the precinct shapefile:  441
Is the ID unique in the election dataset? (T/F)  True
Length of VEST dataset:  441
Number of unique IDs in the VEST dataset:  441
Is the ID unique in the election dataset? (T/F)  True
Difference in length of precinct shapefile and election data:  120
Is the difference between the two datasets the same as missing values? (T/F) True
{'30- Early Voting ', '40- Absentee ', '11- Early Voting ', '30- Question ', '5- Early Voting ', '11- Question ', '8- Absentee ', '10- Question ', '37- Early Voting ', '38- Early Voting ', '37- Question ', '29- Question ', '1- Question ', '28- Absentee ', '39- Absentee ', '35- Early Voting ', '7- Absentee ', '22- Question ', '34- Early Voting ', '4- Question ', '35- Question ', '16- Early Voting ', '12

### Assign early/absentee/question votes

In [22]:
'''This function takes in the original and rounded values and returns the error rate.
Inputs: actual and rounded (int)
Output: Error'''
def error_gen(actual, rounded):
    divisor = sqrt(1.0 if actual < 1.0 else actual)
    return abs(rounded - actual) ** 2 / divisor

'''This function take in a list of votes as floats and the target sum and then returns the rounded list.
Inputs: list of floats, target sum
Outputs: rounded list (int) to target sum'''
def round_to_target(float_vote, target_sum):
    if all(v == 0 for v in float_vote) and target_sum != 0:
        float_vote[0] = target_sum
    n = len(float_vote)
    rounded = [int(x) for x in float_vote]
    up_count = int(target_sum) - sum(rounded)
    errors = [(error_gen(float_vote[i], rounded[i] + 1) - error_gen(float_vote[i], rounded[i]), i) for i in range(n)]
    rank = sorted(errors)
    for i in range(up_count):
        rounded[rank[i][1]] += 1
    return list(rounded)

In [23]:
'''This function retrieves a list of all specified geographies in a dataframe.
Input: pandas dataframe
Output: list of parishes'''
def hd_list(df,geog_field):
    #Get list of all unique parishes in the dataset.
    geog = df[geog_field].unique()
    #print(geog_field, ' list', geog)
    return geog

'''This function creates a dictionary of geography names (key) and the dataframe subsetted to precincts in that geography (value). 
Input: pandas dataframe of election results
Output: Dictionary where keys are geography names and values are dataframes where the geography field value is the key.'''
def subset_hd(df,geog_field):
    geog_listing = hd_list(df, geog_field)
    geog_dict = {}
    for g in geog_listing:
        sub_df = df[df[geog_field] == g]
        mini_dict = {g:sub_df}
        geog_dict.update(mini_dict)
    return geog_dict

'''This function assigns the early vote to each precinct based on respective candidate performance within a geography.
Input: pandas dataframe of election results.
Output: pandas dataframe of election results with early vote assigned to precincts within a geography, and the early vote precincts are removed.'''
def get_early_vote(df,geog_field,not_a_vote_col, prec_field, early_desc_ids):
    geog_dict = subset_hd(df,geog_field) #returns dictionary of each parish and the subsetted df of that parish
    new_geog_dfs = []
    for k,v in geog_dict.items():
        early_vote_rows = v[v[prec_field].isin(early_desc_ids)] #subsets Early Vote ward in the parish df
        #display(early_vote_rows)
        tot_votes = []
        for i in v.columns:
            if i not in not_a_vote_col:
                v[i] = v[i].astype(int) #make sure the election results column is in integer
                col_votes_sum = v[i].sum()
                tot_votes.append(col_votes_sum)
                non_inperson_value = early_vote_rows[i].sum()
                sum_inperson = col_votes_sum-non_inperson_value #sum all of the in-person votes in the parish
                #for each row in the parish, divide the candidate performance by the total in person vote, and then multiply that factor by the early_vote_value and add that to the original vote 
                v[i] = v[i].apply(lambda x: ((x/sum_inperson)*(non_inperson_value)) + x)
                    #identiy all in-person precincts
        in_person_rows = v[~v[prec_field].isin(early_desc_ids)].copy()
        #reassign v(the parish df) to only the in-person precincts, now that the Early Vote as been assigned
        v = in_person_rows
        col_index = 0
        for i in list(v.columns):
            if i not in not_a_vote_col:
                col_tot = tot_votes[col_index]
                v[i] = v[i].fillna(0).astype(np.int64, errors='ignore')
                new_values = round_to_target(list(v[i]),col_tot)
                v[i] = new_values
                col_index = col_index+1
        new_geog_dfs.append(v)
    #once the loop is done, concatonate all of the new dataframes into one dataframe
    df = pd.concat(new_geog_dfs)
    #return the df with early votes assigned and the early vote precincts removed (should reduce the overall row count by 64, as there are 64 parishes LA)
    return df

In [24]:
# Set up parameters to run early vote function

not_a_vote_col = ['','NAME','HD','DISTRICT','DESCRIPTION'] #columns that are not election results
early_desc_ids = ['Question','Absentee','Early Voting']
geog_field = 'HD'
prec_field = 'DESCRIPTION'

In [25]:
# Run early vote function

elec_assigned = get_early_vote(piv,geog_field,not_a_vote_col, prec_field, early_desc_ids)

elec_assigned.head()

<ipython-input-23-35d5e4df2ba4>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v[i] = v[i].astype(int) #make sure the election results column is in integer
<ipython-input-23-35d5e4df2ba4>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v[i] = v[i].apply(lambda x: ((x/sum_inperson)*(non_inperson_value)) + x)


race_cand,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME,DISTRICT,HD,DESCRIPTION
0,01-446 Aurora,631,620,36,25,0,679,14,611,01-446 Aurora,01-446,01,446 Aurora
1,01-455 Fairbanks No. 1,102,121,3,6,0,104,0,128,01-455 Fairbanks No. 1,01-455,01,455 Fairbanks No. 1
2,01-465 Fairbanks No. 2,196,202,15,8,1,224,1,198,01-465 Fairbanks No. 2,01-465,01,465 Fairbanks No. 2
3,01-470 Fairbanks No. 3,400,295,20,20,5,420,1,325,01-470 Fairbanks No. 3,01-470,01,470 Fairbanks No. 3
4,01-475 Fairbanks No. 4,191,159,11,12,1,215,3,151,01-475 Fairbanks No. 4,01-475,01,475 Fairbanks No. 4


In [26]:
print('Election data row count prior to assigning early vote: ', str(piv.shape[0]))
print('Election data row count after assigning early vote: ', str(elec_assigned.shape[0]))
match = str(elec_assigned.shape[0] == p.shape[0])
print('Does the election data now have one less row than the precincts shapefile? (T/F)', match)

Election data row count prior to assigning early vote:  561
Election data row count after assigning early vote:  441
Does the election data now have one less row than the precincts shapefile? (T/F) True


In [27]:
# Assign the votes from the overseas subset to all precincts in the state

tot_votes = []
for i in elec_assigned.columns:
    if i not in not_a_vote_col:
        elec_assigned[i] = elec_assigned[i].astype(int) #make sure the election results column is in integer
        col_votes_sum = elec_assigned[i].sum()
        tot_votes.append(col_votes_sum)
        non_overseas_value = overseas[i].sum()
        elec_assigned[i] = elec_assigned[i].apply(lambda x: ((x/col_votes_sum)*(non_overseas_value)) + x)
        col_index = 0
# Apply largest remainder method to make sure votes sum correctly     
for i in list(elec_assigned.columns):
    if i not in not_a_vote_col:
        col_tot = tot_votes[col_index]
        elec_assigned[i] = elec_assigned[i].fillna(0).astype(np.int64, errors='ignore')
        new_values = round_to_target(list(elec_assigned[i]),col_tot)
        elec_assigned[i] = new_values
        col_index = col_index+1

# Join the election results with the precinct shapefile

In [28]:
join = pd.merge(p,elec_assigned,on='DISTRICT',how='outer',indicator=True)
join.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME_x,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME_y,HD,DESCRIPTION,_merge
0,173,1.565101,11-075,1.0,None,11-075 Palmer City No. 2,2460,1622.0,838.0,0.516646,"POLYGON ((-149.13365 61.61026, -149.13361 61.6...",11-075 Palmer City No. 2,354,654,28,14,3,408,5,631,11-075 Palmer City No. 2,11,075 Palmer City No. 2,both
1,181,286.270233,12-220,1.0,None,12-220 Butte,3990,1622.0,2368.0,1.459926,"POLYGON ((-148.52103 61.62682, -148.52020 61.6...",12-220 Butte,527,1443,38,30,1,588,12,1426,12-220 Butte,12,220 Butte,both
2,182,3.675227,10-055,1.0,None,10-055 Lakes No. 2,1325,1622.0,-297.0,-0.183107,"POLYGON ((-149.33012 61.61391, -149.33229 61.6...",10-055 Lakes No. 2,155,453,19,6,3,177,1,450,10-055 Lakes No. 2,10,055 Lakes No. 2,both
3,32,29.413515,15-300,1.0,None,15-300 JBER No. 2,5937,1622.0,4315.0,2.660296,"POLYGON ((-149.93126 61.25090, -149.92104 61.2...",15-300 JBER No. 2,112,201,7,13,1,161,2,169,15-300 JBER No. 2,15,300 JBER No. 2,both
4,144,6.259140,28-110,1.0,None,28-110 Rabbit Creek No. 2,2785,1622.0,1163.0,0.717016,"POLYGON ((-149.83082 61.07593, -149.83055 61.0...",28-110 Rabbit Creek No. 2,792,1016,15,16,0,812,4,1030,28-110 Rabbit Creek No. 2,28,110 Rabbit Creek No. 2,both


In [29]:
# See what did *not* merge on the unique ID

false = join[join['_merge']!='both'].copy()
join.drop(columns = ['_merge'],inplace=True)
display(false)

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME_x,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry,,G18GOVDBEG,G18GOVRDUN,G18GOVLTOI,G18GOVIWAL,G18GOVOWRI,G18HALDGAL,G18HALOWRI,G18HALRYOU,NAME_y,HD,DESCRIPTION,_merge


# Validate the data with the VEST file

In [30]:
# Join the joined file with the VEST file
test = pd.merge(vest,join,on='DISTRICT',how='outer',indicator=True)
test.head()

,DISTRICT,NAME,G18GOVRDUN_x,G18GOVDBEG_x,G18GOVLTOI_x,G18GOVIWAL_x,G18GOVOWRI_x,G18HALRYOU_x,G18HALDGAL_x,G18HALOWRI_x,geometry_x,ID,AREA,MEMBERS,LOCKED,NAME_x,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry_y,,G18GOVDBEG_y,G18GOVRDUN_y,G18GOVLTOI_y,G18GOVIWAL_y,G18GOVOWRI_y,G18HALDGAL_y,G18HALOWRI_y,G18HALRYOU_y,NAME_y,HD,DESCRIPTION,_merge
0,11-075,11-075 Palmer City No. 2,655,352,28,14,3,632,407,5,"POLYGON ((-149.13365 61.61026, -149.13361 61.6...",173,1.565101,1.0,None,11-075 Palmer City No. 2,2460,1622.0,838.0,0.516646,"POLYGON ((-149.13365 61.61026, -149.13361 61.6...",11-075 Palmer City No. 2,354,654,28,14,3,408,5,631,11-075 Palmer City No. 2,11,075 Palmer City No. 2,both
1,12-220,12-220 Butte,1442,528,37,30,1,1426,588,13,"POLYGON ((-148.52103 61.62682, -148.52020 61.6...",181,286.270233,1.0,None,12-220 Butte,3990,1622.0,2368.0,1.459926,"POLYGON ((-148.52103 61.62682, -148.52020 61.6...",12-220 Butte,527,1443,38,30,1,588,12,1426,12-220 Butte,12,220 Butte,both
2,10-055,10-055 Lakes No. 2,453,155,19,6,3,450,178,1,"POLYGON ((-149.33012 61.61391, -149.33229 61.6...",182,3.675227,1.0,None,10-055 Lakes No. 2,1325,1622.0,-297.0,-0.183107,"POLYGON ((-149.33012 61.61391, -149.33229 61.6...",10-055 Lakes No. 2,155,453,19,6,3,177,1,450,10-055 Lakes No. 2,10,055 Lakes No. 2,both
3,15-300,15-300 JBER No. 2,201,112,8,12,1,170,161,2,"POLYGON ((-149.93126 61.25090, -149.92104 61.2...",32,29.413515,1.0,None,15-300 JBER No. 2,5937,1622.0,4315.0,2.660296,"POLYGON ((-149.93126 61.25090, -149.92104 61.2...",15-300 JBER No. 2,112,201,7,13,1,161,2,169,15-300 JBER No. 2,15,300 JBER No. 2,both
4,28-110,28-110 Rabbit Creek No. 2,1016,792,14,17,0,1030,812,4,"POLYGON ((-149.83082 61.07593, -149.83055 61.0...",144,6.259140,1.0,None,28-110 Rabbit Creek No. 2,2785,1622.0,1163.0,0.717016,"POLYGON ((-149.83082 61.07593, -149.83055 61.0...",28-110 Rabbit Creek No. 2,792,1016,15,16,0,812,4,1030,28-110 Rabbit Creek No. 2,28,110 Rabbit Creek No. 2,both


In [31]:
# See what was *not* successfully joined on the unique ID between the two files.

false = test[test['_merge']!='both'].copy()
display(false)

,DISTRICT,NAME,G18GOVRDUN_x,G18GOVDBEG_x,G18GOVLTOI_x,G18GOVIWAL_x,G18GOVOWRI_x,G18HALRYOU_x,G18HALDGAL_x,G18HALOWRI_x,geometry_x,ID,AREA,MEMBERS,LOCKED,NAME_x,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry_y,,G18GOVDBEG_y,G18GOVRDUN_y,G18GOVLTOI_y,G18GOVIWAL_y,G18GOVOWRI_y,G18HALDGAL_y,G18HALOWRI_y,G18HALRYOU_y,NAME_y,HD,DESCRIPTION,_merge


## Validate election totals

In [32]:
def sum_checker (df, column_List):
    """Checks the sums of different columns and prints out the maximum difference by column. 
    Assumes that the dataframe is the merged dataframe between RDH and the partner, with the same named election result
    columns, except RDH is _x and Partner is _y. 
    """
    list_max = 0
    for i in column_List:
        left_sum = df[i+"_x"].sum()
        right_sum = df[i+"_y"].sum()
        if (left_sum==right_sum):
            print("Column: ", i, " have the same sums")
            
        else:
            diff = abs(left_sum-right_sum)
            percent = abs(left_sum-right_sum)/left_sum
            print("Column: ", i)
            print("RDH sum is ", str(left_sum))
            print("Partner sum is ", str(right_sum))
            print("Their (absolute) difference is ",str(diff))
            print("As a percentage of DF1 this is ", str(round(percent*100,5)) + "%")
            if (percent>list_max):
                list_max = percent
            print("")
    print("The max difference (as a percent of the total of one DF column) is: ", str(round(list_max,5)))

In [33]:
# Create a column list of all election results

col_list = []
for i in vest.columns:
    if '18' in i:
        col_list.append(i)
        
print(col_list)

['G18GOVRDUN', 'G18GOVDBEG', 'G18GOVLTOI', 'G18GOVIWAL', 'G18GOVOWRI', 'G18HALRYOU', 'G18HALDGAL', 'G18HALOWRI']


In [34]:
sum_checker(test,col_list)

Column:  G18GOVRDUN  have the same sums
Column:  G18GOVDBEG  have the same sums
Column:  G18GOVLTOI  have the same sums
Column:  G18GOVIWAL  have the same sums
Column:  G18GOVOWRI  have the same sums
Column:  G18HALRYOU
RDH sum is  149779
Partner sum is  149772
Their (absolute) difference is  7
As a percentage of DF1 this is  0.00467%

Column:  G18HALDGAL
RDH sum is  131199
Partner sum is  131088
Their (absolute) difference is  111
As a percentage of DF1 this is  0.0846%

Column:  G18HALOWRI  have the same sums
The max difference (as a percent of the total of one DF column) is:  0.00085


## Validate the results by precinct

In [35]:
def validater_row (df, column_List,match_col):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    diff_geoids = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                diff_values.append(abs(diff))
                if diff>2:
                    diff_geoids.append(df.iloc[j][match_col])
                    print('Different value for ', i, ' is ', diff, ' for ', df.iloc[j][match_col])
                else:
                    continue
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j][match_col])
                if (diff>max_diff):
                    print("New max diff is: ", str(max_diff))
                    max_diff = diff
                    print(df.iloc[j][match_col])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j][match_col])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences of greater than two votes.")
    print(matching_rows," of these rows are the same or less than two votes difference.")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    print("The median difference is: ", str(np.median(diff_values)))
    return list(set(diff_geoids))

In [36]:
diff_ids = validater_row(test,col_list,'DISTRICT')

Different value for  G18HALDGAL  is  3  for  04-230
New max diff is:  0
04-230
There are  441  total rows
1  of these rows have election result differences of greater than two votes.
440  of these rows are the same or less than two votes difference.

The max difference between any one shared column in a row is:  3
The average difference is:  1.0680028632784537
The median difference is:  1.0


In [37]:
#Subset the joined dataset by those with mismatching election values

not_matching = test[test['DISTRICT'].isin(diff_ids)].copy()
print('\nPrecinct with mismatching data are: ', ', '.join(sorted(list(not_matching['NAME_y'].unique()))))
print('\nThere are ', len(not_matching), ' wards that have at least one mismatched election result by greater than two votes.')
display(not_matching)


Precinct with mismatching data are:  04-230 Ester 

There are  1  wards that have at least one mismatched election result by greater than two votes.


,DISTRICT,NAME,G18GOVRDUN_x,G18GOVDBEG_x,G18GOVLTOI_x,G18GOVIWAL_x,G18GOVOWRI_x,G18HALRYOU_x,G18HALDGAL_x,G18HALOWRI_x,geometry_x,ID,AREA,MEMBERS,LOCKED,NAME_x,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,geometry_y,,G18GOVDBEG_y,G18GOVRDUN_y,G18GOVLTOI_y,G18GOVIWAL_y,G18GOVOWRI_y,G18HALDGAL_y,G18HALOWRI_y,G18HALRYOU_y,NAME_y,HD,DESCRIPTION,_merge
257,04-230,04-230 ESTER,192,471,18,11,0,196,497,4,"POLYGON ((-148.18238 64.90510, -148.18150 64.9...",282,109.095596,1.0,None,04-230 ESTER,1131,1622.0,-491.0,-0.302713,"POLYGON ((-148.18238 64.90510, -148.18150 64.9...",04-230 Ester,470,192,18,11,0,494,4,195,04-230 Ester,04,230 Ester,both


In [38]:
## Validate the geometries

def validate_geom(gdf1,gdf2,sort_field,dec=0):
    gdf1["geometry"]=gdf1.buffer(0) #buffer by 0 to remove any non polygons
    gdf2["geometry"]=gdf2.buffer(0)
    proj = gdf1.crs #project to same projection
    gdf2 = gdf2.to_crs(proj)
    gdf1 = gdf1.to_crs(proj)
    gdf1 = gdf1.sort_values(by=[sort_field]) #sort values by the unique field
    gdf2 = gdf2.sort_values(by=[sort_field])
    gdf2.reset_index(drop=True,inplace=True) #drop the index
    gdf1.reset_index(drop=True,inplace=True)
    ###Add in print link difference between the two files
    ## Look at GA example
    #see the difference between the precinct and final file
    ngdf1 = gdf1.to_crs(proj)
    ngdf2 = gdf2.to_crs(proj)
    ngdf1 = ngdf1.buffer(0)
    ngdf2 = ngdf2.buffer(0)
    file = ngdf2.difference(ngdf1)
    try: 
        print('Difference between gdf1 and gdf2 as a percent of gdf1 total area ', str((sum(file.area)/sum(ngdf1.area))*100))
    except ZeroDivisionError:
        print('Difference between gdf1 and gdf2 as a percent of gdf1 total area is 0.')
    try:
        print('Difference between gdf1 and gdf2 as a percent of gdf2 total area ', str((sum(file.area)/sum(ngdf2.area))*100))
    except ZeroDivisionError:
        print('Difference between gdf1 and gdf2 as a percent of gdf2 total area is 0.')
    #To 6 decimal places, the two files are equal 
    t_f_geom = list(gdf1.geom_almost_equals(gdf2,decimal=dec))
    gdf1['GEOM_EQUALS'] = t_f_geom
    gdf2['GEOM_EQUALS'] = t_f_geom
    gdf1_not_match = gdf1[gdf1['GEOM_EQUALS']==False]
    gdf2_not_match = gdf2[gdf2['GEOM_EQUALS']==False]
    gdf1_not_match = gdf1_not_match.to_crs(proj)
    gdf2_not_match = gdf1_not_match.to_crs(proj)
    gdf1_not_match.buffer(0)
    gdf2_not_match.buffer(0)
    file2 = gdf1_not_match.difference(gdf2_not_match)
    try:
        print('Difference between not matching precincts in gdf1 and not matching precincts in gdf2 as a percent of gdf1 not matching precincts total area ', str((sum(file2.area)/sum(gdf1_not_match.area))*100))
    except ZeroDivisionError:
        print('Difference between not matching precincts in gdf1 and not matching precincts in gdf2 as a percent of gdf1 not matching precincts total area is 0.')
    try:
        print('Difference between not matching precincts in gdf2 and not matching precincts in gdf1 as a percent of gdf2 not matching precincts total area ', str((sum(file2.area)/sum(gdf2_not_match.area))*100))
    except ZeroDivisionError:
        print('Difference between not matching precincts in gdf2 and not matching precincts in gdf1 as a percent of gdf2 not matching precincts total area is 0.')
    unique_geom = []
    for i in t_f_geom:
        if i not in unique_geom:
            unique_geom.append(i)
    for i in unique_geom:
        percent = str((t_f_geom.count(i)/len(t_f_geom))*100) 
        print(str(t_f_geom.count(i)) + ' precincts are ' + str(i) + ' which is ' + percent + ' percent of all precincts.')
    return gdf1, gdf2

In [39]:
geom_validate1 = validate_geom(vest,join,'DISTRICT')

Difference between gdf1 and gdf2 as a percent of gdf1 total area  0.0
Difference between gdf1 and gdf2 as a percent of gdf2 total area  0.0
Difference between not matching precincts in gdf1 and not matching precincts in gdf2 as a percent of gdf1 not matching precincts total area is 0.
Difference between not matching precincts in gdf2 and not matching precincts in gdf1 as a percent of gdf2 not matching precincts total area is 0.
441 precincts are True which is 100.0 percent of all precincts.


<ipython-input-38-5f052acb2669>:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print('Difference between gdf1 and gdf2 as a percent of gdf1 total area ', str((sum(file.area)/sum(ngdf1.area))*100))
<ipython-input-38-5f052acb2669>:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print('Difference between gdf1 and gdf2 as a percent of gdf2 total area ', str((sum(file.area)/sum(ngdf2.area))*100))
<ipython-input-38-5f052acb2669>:41: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print('Difference between not matching precincts in gdf1 and not matching precincts in gdf2 as a percent of